# Étape 1 : Analyse des phrases dans le fichier XML
Les fichiers .data.xml contiennent des phrases ou des contextes annotés, tandis que les fichiers .gold.key.txt associent les mots cibles à leurs sens (par exemple, des BabelNet IDs).

### Extrait des fichiers XML : 
Utilisation de la bibliothèque xml.etree.ElementTree pour extraire les phrases et mots annotés : 

In [2]:
# chemin dataset en français
french_data_path = r"C:\Users\pc\Downloads\BIBDA\S3\Analyse de Sentiments et Text Mining\french-data\training_datasets_fr\semcor_fr\semcor_fr.data.xml"

# chemin dataset en anglais
english_data_path = r"C:\Users\pc\Downloads\BIBDA\S3\Analyse de Sentiments et Text Mining\english-data\training_datasets_en\semcor_en\semcor_en.data.xml"

In [3]:
import xml.etree.ElementTree as ET

def parse_data(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    sentences = []

    for text in root.findall('.//text'):  # Parcourt les textes
        for sentence in text.findall('.//sentence'):  # Parcourt les phrases
            words = []
            # Ne garde que les mots annotés comme <instance>
            for instance in sentence.findall('.//instance'):  # Trouve les mots annotés "instance"
                words.append(instance.text)
            sentences.append(" ".join(words))  # Concatène les mots annotés
    return sentences

In [4]:
# Test avec le fichier XML
sentences = parse_data(english_data_path)

print(sentences[:5])  # Affiche les 5 premières phrases

['long been reviewed objectives benefit service program', 'permitted become giveaway program rather has goal improved employee morale consequently increased productivity', 'effort make assess results program', 'measure reduced absenteeism turnover accidents grievances improved quality output', 'set specific objectives employee publication']


# Étape 2 : Association des mots annotés avec leurs identifiants de sens

In [5]:
def parse_gold_keys(file_path):
    gold_keys = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            
            parts = line.split()
            if len(parts) >= 2:
                instance_id = parts[0]
                # Si plusieurs sens sont associés à l'instance, les ajouter dans une liste
                sense_ids = parts[1:]
                gold_keys[instance_id] = sense_ids
            else:
                print(f"Ligne ignorée (format incorrect): {line}")  # Affiche une ligne mal formatée
    return gold_keys

In [6]:
# Test avec le fichier gold key
gold_key_path = r"C:\Users\pc\Downloads\BIBDA\S3\Analyse de Sentiments et Text Mining\English-data\training_datasets_en\semcor_en\semcor_en.gold.key.txt"
gold_keys = parse_gold_keys(gold_key_path)

print(list(gold_keys.items())[:5])  # Affiche les 5 premiers mappings


[('d000.s000.t000', ['bn:00106124a']), ('d000.s000.t001', ['bn:00083181v']), ('d000.s000.t002', ['bn:00092618v']), ('d000.s000.t003', ['bn:00002179n']), ('d000.s000.t004', ['bn:00009904n'])]


# Étape 3 : Associer les phrases avec leurs annotations 
Nous combinons les phrases et les identifiants de sens en utilisant les informations du fichier gold key :

In [7]:
def associate_annotations(file_path, gold_keys):
    tree = ET.parse(file_path)
    root = tree.getroot()
    annotations = []

    for text in root.findall('.//text'):
        for sentence in text.findall('.//sentence'):
            sentence_id = sentence.attrib['id']  # ID de la phrase
            annotated_words = []
            for instance in sentence.findall('.//instance'):  # Parcourt les instances annotées
                instance_id = instance.attrib['id']  # ID de l'instance
                word = instance.text
                sense_id = gold_keys.get(instance_id, "unknown")  # Récupère l'identifiant de sens
                annotated_words.append((word, sense_id))
            annotations.append((sentence_id, annotated_words))
    return annotations

In [8]:
# Associer annotations
annotations = associate_annotations(english_data_path, gold_keys)

# Afficher les 5 premières annotations
for sentence_id, annotated_words in annotations[:5]:
    print(f"Sentence ID: {sentence_id}")
    for word, sense_id in annotated_words:
        print(f"  Word: {word}, Sense ID: {sense_id}")

Sentence ID: d000.s000
  Word: long, Sense ID: ['bn:00106124a']
  Word: been, Sense ID: ['bn:00083181v']
  Word: reviewed, Sense ID: ['bn:00092618v']
  Word: objectives, Sense ID: ['bn:00002179n']
  Word: benefit, Sense ID: ['bn:00009904n']
  Word: service, Sense ID: ['bn:00070654n']
  Word: program, Sense ID: ['bn:00064646n']
Sentence ID: d000.s001
  Word: permitted, Sense ID: ['bn:00082536v']
  Word: become, Sense ID: ['bn:00083294v']
  Word: giveaway, Sense ID: ['bn:00040564n']
  Word: program, Sense ID: ['bn:00064646n']
  Word: rather, Sense ID: ['bn:00115972r']
  Word: has, Sense ID: ['bn:00088078v']
  Word: goal, Sense ID: ['bn:00030721n']
  Word: improved, Sense ID: ['bn:00104710a']
  Word: employee, Sense ID: ['bn:00030618n']
  Word: morale, Sense ID: ['bn:00055852n']
  Word: consequently, Sense ID: ['bn:00114106r']
  Word: increased, Sense ID: ['bn:00104887a']
  Word: productivity, Sense ID: ['bn:00064598n']
Sentence ID: d000.s002
  Word: effort, Sense ID: ['bn:00007011n']
  W

In [9]:
print(len(sentences))
print(len(annotations))

37176
37176


In [10]:
for i in range(5):
    print(f"Phrase : {sentences[i]}")
    print(f"Annotations : {annotations[i]}")

Phrase : long been reviewed objectives benefit service program
Annotations : ('d000.s000', [('long', ['bn:00106124a']), ('been', ['bn:00083181v']), ('reviewed', ['bn:00092618v']), ('objectives', ['bn:00002179n']), ('benefit', ['bn:00009904n']), ('service', ['bn:00070654n']), ('program', ['bn:00064646n'])])
Phrase : permitted become giveaway program rather has goal improved employee morale consequently increased productivity
Annotations : ('d000.s001', [('permitted', ['bn:00082536v']), ('become', ['bn:00083294v']), ('giveaway', ['bn:00040564n']), ('program', ['bn:00064646n']), ('rather', ['bn:00115972r']), ('has', ['bn:00088078v']), ('goal', ['bn:00030721n']), ('improved', ['bn:00104710a']), ('employee', ['bn:00030618n']), ('morale', ['bn:00055852n']), ('consequently', ['bn:00114106r']), ('increased', ['bn:00104887a']), ('productivity', ['bn:00064598n'])])
Phrase : effort make assess results program
Annotations : ('d000.s002', [('effort', ['bn:00007011n']), ('make', ['bn:00087106v']), (

# Resultats

Extraction des données XML : Les phrases ont été correctement extraites à partir du fichier XML, en se concentrant sur les instances annotées.
Gestion des clés dorées (gold keys) : Les clés dorées (celles correspondant aux sens des mots) ont été extraites et plusieurs identifiants de sens sont gérés correctement pour une même instance.

# Prochaine étape : Désambiguïsation des mots avec un modèle Transformers
Maintenant que les données sont correctement extraites et annotées, la prochaine étape serait d'appliquer un modèle de désambiguïsation de sens (Word Sense Disambiguation - WSD) utilisant un modèle de type Transformers.

Pour cela, on peux utiliser des modèles pré-entraînés comme BERT, RoBERTa (version améliorés de BERT), ou XLM-R (version multilingue de BERT) qui sont bien adaptés pour les tâches de désambiguïsation des mots. Ces modèles sont disponibles via la bibliothèque transformers de Hugging Face.

# BERT ?

# RoBERTa ?

#### Conversion en JSON

In [12]:
import json

def export_to_json(sentences_with_annotations, output_path):
    """
    Exporte les données des phrases avec annotations dans un fichier JSON.
    
    :param sentences_with_annotations: Liste contenant des tuples (ID de phrase, annotations).
    :param output_path: Le chemin du fichier de sortie pour le fichier JSON.
    """
    sentences_data = []
    for sentence_data in sentences_with_annotations:
        sentence_id = sentence_data[0]  # L'ID de la phrase se trouve à l'index 0
        annotations = [
            [annotation[0], annotation[1]]  # Le mot est à l'index 0 et les sens à l'index 1
            for annotation in sentence_data[1]
        ]
        sentences_data.append([sentence_id, annotations])
    
    with open(output_path, 'w', encoding='utf-8') as json_file:
        json.dump(sentences_data, json_file, ensure_ascii=False, indent=4)
    print(f"Les données ont été exportées en JSON vers {output_path}")

In [15]:
output_path_json = r"C:\Users\pc\Downloads\BIBDA\S3\Analyse de Sentiments et Text Mining\english-data\english_wsd_dataset.json"
export_to_json(annotations, output_path_json)

Les données ont été exportées en JSON vers C:\Users\pc\Downloads\BIBDA\S3\Analyse de Sentiments et Text Mining\english-data\english_wsd_dataset.json


In [27]:
import pandas as pd
pd.read_json(output_path_json).head(5)

,0,1
0,d000.s000,"[[long, [bn:00106124a]], [been, [bn:00083181v]..."
1,d000.s001,"[[permitted, [bn:00082536v]], [become, [bn:000..."
2,d000.s002,"[[effort, [bn:00007011n]], [make, [bn:00087106..."
3,d000.s003,"[[measure, [bn:00082720v]], [reduced, [bn:0010..."
4,d000.s004,"[[set, [bn:00086468v]], [specific, [bn:0011100..."


#### Conversion en pandas DataFrame

In [28]:
def export_to_dataframe(sentences_with_annotations, output_path):
    """
    Exporte les données des phrases avec annotations dans un fichier CSV.
    
    :param sentences_with_annotations: Liste contenant des tuples (ID de phrase, annotations).
    :param output_path: Le chemin du fichier de sortie pour le fichier CSV.
    """
    data = []
    
    for sentence_data in sentences_with_annotations:
        sentence_id = sentence_data[0]  # L'ID de la phrase se trouve à l'index 0
        for annotation in sentence_data[1]:  # Les annotations sont à l'index 1
            word = annotation[0]  # Le mot est à l'index 0 de l'annotation
            sense_id = ', '.join(annotation[1])  # Les sens sont à l'index 1 et doivent être joints par une virgule
            data.append([sentence_id, word, sense_id])
    
    df = pd.DataFrame(data, columns=['sentence', 'word', 'sense'])
    df.to_csv(output_path, index=False, encoding='utf-8')
    print(f"Les données ont été exportées en CSV vers {output_path}")

In [29]:
output_path_csv = r"C:\Users\pc\Downloads\BIBDA\S3\Analyse de Sentiments et Text Mining\english-data\english_wsd_dataset.csv"
export_to_dataframe(annotations, output_path_csv)

Les données ont été exportées en CSV vers C:\Users\pc\Downloads\BIBDA\S3\Analyse de Sentiments et Text Mining\english-data\english_wsd_dataset.csv


In [33]:
df = pd.read_csv(output_path_csv)
df.head(5)

,sentence,word,sense
0,d000.s000,long,bn:00106124a
1,d000.s000,been,bn:00083181v
2,d000.s000,reviewed,bn:00092618v
3,d000.s000,objectives,bn:00002179n
4,d000.s000,benefit,bn:00009904n


# Vérification des valeurs manquantes

In [34]:
import re
# Vérification des valeurs manquantes
missing_values = df.isnull().sum()

# Vérification des doublons
duplicates = df.duplicated().sum()

In [35]:
# Vérification du format des Sense IDs
def is_valid_sense_id(sense_id):
    pattern = r'^bn:\d{8}[a-z]$'
    return bool(re.match(pattern, sense_id))

invalid_sense_ids = df[~df['sense'].apply(is_valid_sense_id)]

In [36]:
# Résultats
print("Résumé des vérifications :\n")
print(f"Valeurs manquantes par colonne :\n{missing_values}\n")
print(f"Nombre de doublons : {duplicates}\n")
if not invalid_sense_ids.empty:
    print(f"Sense IDs invalides :\n{invalid_sense_ids}\n")
else:
    print("Tous les Sense IDs sont valides.\n")

Résumé des vérifications :

Valeurs manquantes par colonne :
sentence    0
word        1
sense       0
dtype: int64

Nombre de doublons : 5621

Sense IDs invalides :
         sentence      word                       sense
72      d000.s010   service  bn:00070653n, bn:00070652n
371     d000.s046    public  bn:00109212a, bn:00109211a
1102    d001.s006     break  bn:00083902v, bn:00083904v
1159    d001.s016  handling  bn:00084527v, bn:00087116v
1196    d001.s023     cause  bn:00016845n, bn:00016850n
...           ...       ...                         ...
222803  d339.s009    caring  bn:00084527v, bn:00084524v
223324  d340.s122       lay  bn:00090322v, bn:00090321v
224476  d345.s107    called  bn:00084385v, bn:00084386v
224498  d345.s116      call  bn:00084385v, bn:00084386v
224618  d346.s042      show  bn:00083245v, bn:00086557v

[656 rows x 3 columns]



In [40]:
!pip install transformers

DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330



   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.0 MB 8.5 MB/s eta 0:00:02
   -------- ------------------------------- 2.1/10.0 MB 6.9 MB/s eta 0:00:02
   ---------------- ----------------------- 4.2/10.0 MB 7.2 MB/s eta 0:00:01
   ---------------------- ----------------- 5.8/10.0 MB 7.0 MB/s eta 0:00:01
   ---------------------------- ----------- 7.1/10.0 MB 7.0 MB/s eta 0:00:01
   ---------------------------------- ----- 8.7/10.0 MB 7.0 MB/s eta 0:00:01
   ---------------------------------------  10.0/10.0 MB 6.9 MB/s eta 0:00:01
   ---------------------------------------- 10.0/10.0 MB 6.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ----------------- ---------------------- 1.0/2.4 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------  2.4/2.4 MB 6.4 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 5.4 MB/s eta 0:00:00


In [53]:
from transformers import AutoTokenizer

# Charger le tokenizer pour le modèle souhaité
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

In [55]:
def prepare_data_for_model(df, tokenizer, max_len=128):
    # Vérification des données
    print("Aperçu des mots : ", df['word'].head())
    print("Types des mots : ", df['word'].apply(type).unique())

    # Convertir toutes les valeurs en chaînes, si nécessaire
    df['word'] = df['word'].astype(str)

    # Tokenization
    inputs = tokenizer.batch_encode_plus(
        df['word'].tolist(),  # Liste des mots
        max_length=max_len,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    # Retourner les entrées tokenisées
    return inputs

In [56]:
inputs = prepare_data_for_model(df, tokenizer)

Aperçu des mots :  0          long
1          been
2      reviewed
3    objectives
4       benefit
Name: word, dtype: object
Types des mots :  [<class 'str'>]


In [60]:
print(inputs[:20])  # Aperçu des deux premières phrases

[Encoding(num_tokens=128, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=128, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=128, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=128, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=128, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=128, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=128, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=128, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=128, attrib